In [1]:
from datetime import datetime as dt
from datetime import timedelta
import logging
import importlib
import urllib
import unicodecsv as csv
from lxml import html
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import sys
from requests.auth import HTTPProxyAuth
import time
import os

In [2]:
#Script is written right now for before cleaned, single region. Must adapt for cleaned and multi-region.
#This will include changing "price" to "rent"
#QUESTION: Unclear what is actually being counted in 'score' variable

cleandf = pd.read_csv('C:\\Users\\james\\Documents\\Berkeley_Docs\\Spring_17_Courses\\CP290 Data Lab\\github\\scraper2\\shared_room_scraper\\sfbay_merged_2017_04_10.csv',header=0)
#determine rent per square foot


In [3]:
cleandf.shape

(30231, 21)

In [4]:
cleandf.loc[cleandf.lat == 99, 'lat'] = 0
cleandf.loc[cleandf.lng == 99, 'lng'] = 0
cleandf.loc[cleandf.sqft == 0, 'sqft'] = np.nan

In [5]:
#For any duplicate post, we want to keep the version with the most information. Therefore, assign a score to each post and 
#give one point for square footage, lat, long and price
#OLD FORMULA
#cleandf['score']=cleandf.astype(bool).sum(axis=1)

cleandf['price_exists'] = cleandf['price']>0
cleandf['sqft_exists'] = cleandf['sqft']>0
cleandf['lat_exists'] = cleandf['lat']>0
cleandf['lng_exists'] = cleandf['lng']<0
cleandf['score'] = cleandf[['price_exists','sqft_exists','lat_exists','lng_exists']].astype(bool).sum(axis=1)

In [6]:
#Sort rows by score
cleandf = cleandf.sort_values(by='score',ascending=False)

In [7]:
cleandf.shape

(30231, 26)

In [8]:
cleandf.tail()

,pid,dt,url,title,price,neighb,sqft,lat,lng,accuracy,...,private_room,bath_known,private_bath,parking_known,onsite_parking,price_exists,sqft_exists,lat_exists,lng_exists,score
7347,6065495036,2017-04-02 13:32,http://sfbay.craigslist.org/sfc/roo/6065495036...,$800 - OPEN BEDROOM IN LARGE MARINA APARTMENT,NaN,marina / cow hollow,NaN,0.0,0.0,99,...,True,True,False,True,False,False,False,False,False,0
20308,6075917680,2017-04-05 15:46,http://sfbay.craigslist.org/pen/roo/6075917680...,large room set up like studio,NaN,east palo alto,NaN,0.0,0.0,99,...,True,True,True,False,False,False,False,False,False,0
20364,6075830704,2017-04-05 15:09,http://sfbay.craigslist.org/sfc/roo/6075830704...,$620 - LOOKING FOR INDIVIDUAL TO RENT LARGE MA...,NaN,SOMA / south beach,NaN,0.0,0.0,99,...,True,True,True,True,True,False,False,False,False,0
7322,6053205700,2017-04-02 11:10,http://sfbay.craigslist.org/sfc/roo/6053205700...,male looking for female roommate,NaN,orange county,NaN,0.0,0.0,99,...,False,True,True,True,False,False,False,False,False,0
20295,6037956801,2017-04-05 15:55,http://sfbay.craigslist.org/sfc/roo/6037956801...,雅房 Mission夹Geneva,NaN,ingleside / SFSU / CCSF,NaN,0.0,0.0,99,...,True,True,False,False,False,False,False,False,False,0


In [9]:
#List as 'true' if the PID, body_text, or the title are duplicates
cleandf['body_text_duplicate']=cleandf.duplicated('body_text')
cleandf['title_duplicate']=cleandf.duplicated('title')
cleandf['PID_duplicate']=cleandf.duplicated('pid')


In [10]:
cleandf_unique = cleandf.drop_duplicates('pid')
cleandf_unique.shape

(12434, 29)

In [11]:
#Keep rows with highest score. Drop rows with duplicate PIDs, Duplicate Titles, Duplicate Posting Bodies
cleandf_unique = cleandf_unique.drop_duplicates('body_text')
cleandf_unique.shape

(9743, 29)

In [12]:
cleandf_unique = cleandf_unique.drop_duplicates('title')
cleandf_unique.shape

(8320, 29)

## Creating new fields for analysis

In [13]:
cleandf_unique['dt'] = pd.to_datetime(cleandf_unique['dt'], format='%Y-%m-%d')
cleandf_unique['region'] = cleandf_unique['url'].str.extract('http://(.*).craigslist.org', expand=False)
cleandf_unique['day_of_week'] = cleandf_unique['dt'].apply(lambda x: x.weekday())
cleandf_unique['rent_sqft'] = cleandf_unique['price'] / cleandf['sqft']

In [14]:
#counting listings per day (right now inlcudes time, need to remove)
listings_per_date = cleandf_unique['dt'].value_counts()

## Descriptive Stats

In [15]:
cleandf_unique.describe()

,pid,price,sqft,lat,lng,accuracy,score,day_of_week,rent_sqft
count,8.320000e+03,7.538000e+03,2858.000000,8320.000000,8320.000000,8320.000000,8320.000000,8320.000000,2818.000000
mean,6.070679e+09,1.166763e+04,711.120364,32.419325,-105.210819,25.763341,2.971394,3.292788,55.358807
std,1.031722e+07,3.449174e+05,2525.405332,13.033776,42.290405,30.102703,0.976044,2.102059,1860.755512
min,6.025795e+09,1.000000e+00,1.000000,0.000000,-123.195100,0.000000,0.000000,0.000000,0.000500
25%,6.068551e+09,8.000000e+02,140.000000,37.333582,-122.417501,7.000000,3.000000,1.000000,1.318994
50%,6.073157e+09,1.015000e+03,223.500000,37.687300,-122.151060,15.000000,3.000000,4.000000,4.500000
75%,6.077527e+09,1.350000e+03,1025.250000,37.785232,-121.896607,22.000000,4.000000,5.000000,7.098171
max,6.082804e+09,1.300150e+07,90495.000000,39.544513,0.000000,99.000000,4.000000,6.000000,85723.928571


In [16]:
unique_wprice = cleandf_unique[cleandf_unique['price'] > 0]

In [17]:
# in this cell, define the values by which we will filter the 3 columns
upper_percentile = 0.995
lower_percentile = 0.005

# how many rows would be within the upper and lower percentiles?
upper = int(len(unique_wprice) * upper_percentile)
lower = int(len(unique_wprice) * lower_percentile)

# get the rent/sqft values at the upper and lower percentiles
rent_sqft_sorted = unique_wprice['rent_sqft'].sort_values(ascending=True, inplace=False)
upper_rent_sqft = rent_sqft_sorted.iloc[upper]
lower_rent_sqft = rent_sqft_sorted.iloc[lower]

# get the rent values at the upper and lower percentiles
rent_sorted = unique_wprice['price'].sort_values(ascending=True, inplace=False)
upper_rent = rent_sorted.iloc[upper]
lower_rent = rent_sorted.iloc[lower]

# get the sqft values at the upper and lower percentiles
sqft_sorted = unique_wprice['sqft'].sort_values(ascending=True, inplace=False)
upper_sqft = sqft_sorted.iloc[upper]
lower_sqft = sqft_sorted.iloc[lower]

print('valid rent_sqft range:', [lower_rent_sqft, upper_rent_sqft])
print('valid rent range:', [lower_rent, upper_rent])
print('valid sqft range:', [lower_sqft, upper_sqft])

valid rent_sqft range: [0.28000000000000003, nan]
valid rent range: [50.0, 3100.0]
valid sqft range: [70.0, nan]


In [18]:
#Filtering out rows with unreasonable rent prices 
rent_mask = (unique_wprice['price'] > lower_rent) & (unique_wprice['price'] < upper_rent)

In [19]:
filtered_listings = pd.DataFrame(unique_wprice[rent_mask])

In [20]:
filtered_listings.describe()

,pid,price,sqft,lat,lng,accuracy,score,day_of_week,rent_sqft
count,7.460000e+03,7460.000000,2787.000000,7460.000000,7460.000000,7460.000000,7460.000000,7460.000000,2787.000000
mean,6.070542e+09,1114.439678,705.411912,32.773381,-106.357986,24.873056,3.114075,3.268499,5.957566
std,1.040872e+07,422.441018,2553.076966,12.659021,41.072908,29.304104,0.869708,2.103732,23.757866
min,6.025795e+09,55.000000,1.000000,0.000000,-123.195100,0.000000,1.000000,0.000000,0.005470
25%,6.068421e+09,800.000000,140.000000,37.343041,-122.418006,7.000000,3.000000,1.000000,1.332593
50%,6.073091e+09,1015.000000,220.000000,37.700932,-122.178911,15.000000,3.000000,4.000000,4.519231
75%,6.077458e+09,1350.000000,1012.000000,37.785726,-121.911679,22.000000,4.000000,5.000000,7.110324
max,6.082804e+09,3000.000000,90495.000000,39.544513,0.000000,99.000000,4.000000,6.000000,950.000000


In [ ]:
plt.figure(1, figsize=(10,6), )
plt.suptitle('Rent Price Distribution')
plt.xlabel('Price')
plt.ylabel('Number of Listings')
ax = plt.hist((filtered_listings['price']), bins=25, alpha=.4, color='cyan')
plt.show()

In [ ]:
#filtered_listings.to_csv('sfbay_filtered4_10.csv')

In [ ]:
# calculate the ratios of unique to duplicate listings for each region
listings_ratios = pd.DataFrame()

# number of total listings for each region
listings_ratios['cleandf'] = cleandf['region'].value_counts()

# number of duplicate listings for each region (ie, listings that share a pid with at least one other listing)
listings_ratios['duplicate_listings'] = duplicate_listings['region'].value_counts()

# number of unique listings for the region (ie, none share a pid with another listing)
listings_ratios['unique_listings'] = unique_listings['region'].value_counts()

# percent of this region's listings that are duplicates
listings_ratios['duplicate_ratio'] = listings_ratios['duplicate_listings'] / listings_ratios['all_listings']

# percent of this region's listings that are unique (ie, not duplicates)
listings_ratios['unique_ratio'] = listings_ratios['unique_listings'] / listings_ratios['all_listings']

listings_ratios.head()